In [ ]:
# General Use
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Clustering
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# 3D Visualization 
import plotly as py
import plotly.graph_objs as go

In [ ]:
# Carga de datos
df = pd.read_csv("Mall_Customers.csv", index_col="CustomerID")
df.head()

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df.drop(columns=['Gender'], inplace=True)# eliminamos la columna genero

In [ ]:
sns.distplot(df["Age"])# grafica de la distribucion de la edad

In [ ]:
sns.distplot(df["Annual Income (k$)"])# grafica de la distribucion de la ingreso

In [ ]:
sns.distplot(df["Spending Score (1-100)"])# grafica de la distribucion de la puntuacion

### K-means Clustering

K-means is one of the most known algorithms to find clusters, works with large datasets while being fast and efficient. This technique was developed in 1957!

- Centroid-based clustering algorithm
- Maximize intra-cluster similarity
- Minimize inter-Cluster similarity

To start we need to set the number of clusters for the K-Means algorithm - How many clusters do we want? - Usually represented by K.

1. Then, the algorithm will randomly select K centers, one for each cluster of data (which are called centroids)
2. Assign each instance to the closest centroid.
3. Calculate the centroids of each cluster formed
4. Return to step 2
5. End when the centroids no longer change

Randomly selected centroids impact the results ? Yes, but with scikit-learn

### Client Segmentation with Annual Income and Spending Score

In [ ]:
df1 = df[["Annual Income (k$)", "Spending Score (1-100)"]]# creamos una copia de la base de datos 

In [ ]:
df1.plot(kind="scatter",  # tipo de grafica 
        x="Annual Income (k$)",# etiqueta eje x 
        y="Spending Score (1-100)",
        figsize=(10, 7))  # tamano de la grafica
plt.show()

In [ ]:
sum_of_sqr_dist = {} # diccionario para la suma de la distancia al cuadrada

for k in range(1, 10): # bucle para el valor de k numero de grupos
    km = KMeans(n_clusters=k, init='k-means++', max_iter=1000) # k-means de la biblioteca de scikit-learn, definimos n grupos y init inicializa los centroides  
    km = km.fit(df1) # ajuste del modelo
    sum_of_sqr_dist[k] = km.inertia_ # calculo de la suma de la distancia


In [ ]:
sns.pointplot(x=list(sum_of_sqr_dist.keys()), y=list(sum_of_sqr_dist.values())) # grafica de la suma de la distancia al cuadrada 
plt.xlabel("Number of Clusters (K)") # etiqueta eje x 
plt.ylabel("Sum of Square Distances") # etiqueta eje y 
plt.title("Elbow Method for Optimal K") # titulo
plt.show() # mostrar grafica 

In [ ]:
# definimos el modelo
# definimos el numero de grupos
# definimos la inicializacion
# definimos el maximo de iteraciones
Model = KMeans(n_clusters=5, # numero de grupos 
             init='k-means++', # inicializacion
             max_iter=1000) # maximo de iteraciones
Model.fit(df1)

In [ ]:
print("Labels", Model.labels_)

In [ ]:
print("Centroids", Model.cluster_centers_)

In [ ]:
# obtenemos las etiquetas
centroids = Model.cluster_centers_

In [ ]:
# creamos una copia
# agregamos una nueva columna
df1_cluster = df1.copy()
df1_cluster["Cluster"] = Model.fit_predict(df1)

In [ ]:
# verificamos
df1_cluster.head()

In [ ]:
# graficamos
# definimos el color
# agregamos una nueva columna
Color = ['red', 'blue', 'black', 'gold', 'navy']
df1_cluster['Color'] = df1_cluster['Cluster'].map(lambda p:Color[p])

In [ ]:
plt.figure(figsize=(20, 10))
plt.scatter(df1["Annual Income (k$)"],
           df1["Spending Score (1-100)"],
           c=df1_cluster["Color"])

plt.scatter(centroids[:, 0], centroids[:, 1], c='green', s=250)
plt.show()

In [ ]:
# obtenemos las etiquetas
labels = Model.labels_

In [ ]:
# calculamos el coeficiente de silueta
silhouette_score(df1, labels)

In [ ]:
silhouette = {} # Almacenamiento de valores 

for k in range(2,8):# el rango es de 2 a 7
    km = KMeans(n_clusters=k, init='k-means++', max_iter=1000) # k-means de la biblioteca de scikit-learn, definimos n grupos y init inicializa los centroides 
    km.fit(df1) # ajuste del modelo
    silhouette[k] = silhouette_score(df1, km.labels_) # calculo del coeficiente de silueta

In [ ]:
sns.pointplot(x=list(silhouette.keys()), y=list(silhouette.values()))
plt.xlabel("Number of Clusters (K)")
plt.ylabel("Silhouette Scores")
plt.title("Silhouette Scores for Each K")
plt.show()

In [ ]:
df2= df.copy()

In [ ]:
sum_of_sqr_dist = {}

for k in range(1, 10):
    km = KMeans(n_clusters=k, init='k-means++', max_iter=1000)
    km = km.fit(df1)
    sum_of_sqr_dist[k] = km.inertia_

In [ ]:
sns.pointplot(x=list(sum_of_sqr_dist.keys()), y=list(sum_of_sqr_dist.values()))
plt.xlabel("Number of Clusters (K)")
plt.ylabel("Sum of Square Distances")
plt.title("Elbow Method for Optimal K")
plt.show()

In [ ]:
Model2 = KMeans(n_clusters=6, init='k-means++', max_iter=1000)
Model2.fit(df2)

In [ ]:
df2['Cluster'] = Model2.fit_predict(df2)# agregamos una nueva columna 
df2.head()# verificamos

In [ ]:
labels = Model2.labels_# obtenemos las etiquetas 
centroids = Model2.cluster_centers_# obtenemos los centros

In [ ]:
df2['labels'] = labels# agregamos una nueva columna

trace = go.Scatter3d(# definimos el tipo de grafica
                    x = df2['Age'],# agregamos una nueva columna
                    y = df2['Annual Income (k$)'],# agregamos una nueva columna
                    z = df2['Spending Score (1-100)'],# agregamos una nueva columna
                    mode = 'markers',# definimos el tipo de grafica
                    marker=dict(color=df2['labels'], size = 5, line=dict(color=df2['labels'], width=12), opacity=0.8)# definimos el color
                    )
    
data = [trace]# agregamos una nueva columna
layout = go.Layout(# definimos el tipo de grafica
                    title='Clusters',# agregamos una nueva columna
                    scene= dict(# definimos el tipo de grafica
                                xaxis = dict(title = 'Age'),# agregamos una nueva columna
                                yaxis = dict(title = 'Annual Income (k$)'),# agregamos una nueva columna
                                zaxis = dict(title = 'Spending Score (1-100)'))# agregamos una nueva columna
                                )

fig = go.Figure(data=data, layout=layout)# agregamos una nueva columna
py.offline.iplot(fig)#  agregamos una nueva columna
plt.show()# agregamos una nueva columna